Universidad de Chile
Facultad de Ciencias Físicas y Matemáticas
Departamento de Ingeniería Matemática

MA3711-1 Optimización Matemática, 2023-1.      
Profesor: Jorge Amaya.    
Auxiliar: Aldo Gutierrez.     
Ayudante: Carolina Chiu.    
Alumna: Antonia Valenzuela.    

# Tarea Numérica $\#1$

###Parte 1
Para plantear el problema como uno de programación lineal, procedemos como
1. Como se trata de un problema de producción, dejaremos que las variables de decisión correspondan justamente a los cinco productos $P_1 , P_2, P_3, P_4,  P_5$.
2. Definiremos la función objetivo como la utilidad, la cual buscaremos maximizar, es decir, si definimos $P= (P1,P2,P3,P4,P5)^t$ el vector decisión, entonces la función objetivo resulta $c^t P= 1,1P_1 +2,1P_2 + 2,18P_3 + 2,21P_4 + 4,4P_5$ \\
Donde el vector $c^t$ contiene los coeficientes dados por la diferencia de los vectores ganancia y costo por unidad.
3. Estudiemos cada una de las restricciones
  - Se nos dice que ningún producto debe ser producido más del doble del otro, lo cual se puede traducir como $\forall i \neq j, P_i \leq 2P_j$ donde $ i,j = 1,...,5.$ 
  - Se tiene un presupuesto de 5000, por tanto el vector de costos por unidad multiplicado por el vector decisión satisface $4,14P_1 + 5,2P_2 +6,66P_3 +1,97P_4+ 5,03P_5 \leq 5000 $.
  - Se tiene también una restricción temporal sobre las máquinas $M_1, M_2,M_3,M_4$ y $M_5$. Por ejemplo, para producir una unidad de $P_1$, se requieren 1,5 horas semanales de $M_1$, 1 hora de $M_2$, 1,5 horas de $M_3$ y 2 horas de $M_4$. Podemos formular, de esta manera \\

  $$ \begin{bmatrix}1,5 & 1 & 2,4 & 1 & 3 \\
1 & 5 & 1 & 3,5 & 0,5  \\
1,5 & 3  & 3,5  & 1 & 2  \\ 
2 & 4 & 4,2 & 4,3 & 6 
\end{bmatrix}
\begin{bmatrix} P_{1} \\
P_{2} \\
P_{3} \\
P_{4} \\
P_{5}
\end{bmatrix} 
\leq 
\begin{bmatrix} 2000 \\
8000 \\
5000 \\
12000
\end{bmatrix}$$

      Que dará como resultado una matriz de dimensiones 4x1, donde cada fila corresponderá a una restricción para nuestro problema.
 - Luego, intuitivamente vamos a querer que las variables de decisión sean mayores o iguales a 0, esto es $P_{i} \geq 0, \forall i \in \{1,2,3,4,5 \} $

Finalmente, el programa lineal a resolver es \\

$$
 (PL) \left \{ 
 \begin{array}{rcrll}
    \max && 1,1P_{1} + 2,1P_{2} + 2,18P_{3} +2,21P_{4} + 4,4P_{5} \\
    \text{s.a.}  
    &&    4,14P_{1} +5,2P_{2} + 6,66P_{3} + 1,97P_{4} + 5,03P_{5}  &\leq&  5000 \\
    &&    1,5P_{1}+P_{2}+2,4P_{3}+P_{4} + 3P_{5}  &\leq&  2000 \\
    &&    P_{1}+ 5P_{2} + P_{3} + 3,5P_{4} + 0,5P_{5} &\leq& 8000 \\
    &&    1,5P_{1}+3P_{2} + 3,5P_{3} + P_{4}+2P_{5} &\leq& 5000 \\
    &&    2P_{1}+4P_{2} + 4,2P_{3}+ 4,3P_{4} + 6P_{5} &\leq& 12000 \\
    &&    P_{i}\leq 2P_{j},\ \forall i \neq j &&\\
    &&    P_{i}\geq 0,\ \forall i=1,\dots,5. &&
 \end{array}
 \right.
 $$

In [73]:
import numpy as np
from scipy.optimize import linprog
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [74]:
c = [-1.1,-2.1,-2.18,-2.21,-4.4] #vector de costos
A = [[4.14,5.2,6.66,1.97,5.03],[1.5,1.0,2.4,1.0,3.0],[1.0,5.0,1.0,3.5,0.5],[1.5,3.0,3.5,1.0,2.0],[2.0,4.0,4.2,4.3,6.0]] #matriz de 5x5

#nos falta definir la restricción P_{i} \leq P_{j} \forall j \neq i
#definamos una lista inicialmente vacía
stock = []
for i in range(5):
    for j in range(5):
      if i !=j:
        p=np.zeros(5)
        p[i]=1
        p[j]=-2 #esto es [0,1,-2,0,0] por ej, lo cual es equivalente a decir 1p_{1}<=2p_{2}
        stock += [p]

#concatenamos 
A= A + stock
#donde la matriz stock tiene dimensiones 20x5 (pues son 20 iteraciones)
#luego, A* P = b, A \in \mathcal(M)_{25x5}, P \in \mathcal(M)_{5x1} entonces b \in mathcal(M)_{25x1}
b= np.zeros((25,1))
#asignamos
b[0]=5000
b[1]=2000
b[2]=8000
b[3]=5000
b[4]=12000

#en las demás filas de b habrán 0's, en consonancia con que cada fila verifique p_{i} - 2p_{j}<= 0

P1_bound=(0, None)
P2_bound=(0, None)
P3_bound=(0, None)
P4_bound=(0, None)
P5_bound=(0, None) 

#como el solver minimiza, utilizamos max k = - min -k = -min c.
sol = linprog(c, A, b, bounds=[P1_bound,P2_bound,P3_bound, P4_bound,P5_bound])

In [75]:
print("La solución es", -sol.fun)
print("Las cantidades óptimas a producir son", sol.x)

La solución es 2963.5922330097083
Las cantidades óptimas a producir son [145.63106796 266.99029126 145.63106796 291.26213592 291.26213592]


La siguiente tabla resume las cantidad óptimas

| Producto | Cantidad óptima a producir | 
|----------|----------|
| $P_{1}$    | 145.63   |
| $P_{2}$    | 266.99   | 
| $P_{3}$   | 145.63   | 
| $P_{4}$ | 291.26 |
|$P_{5}$|  291.26 |




Para discutir las horas usadas por cada máquina en fabricación de cada producto, consideremos el tiempo total consumido por una máquina al fabricar los cinco productos (en el óptimo calculado). Más aún, se puede determinar el tiempo total consumido por una máquina para cada producto:

$$ \begin{pmatrix}1,5 & 1 & 2,4 & 1 & 3 \\
1 & 5 & 1 & 3,5 & 0,5  \\
1,5 & 3  & 3,5  & 1 & 2  \\ 
2 & 4 & 4,2 & 4,3 & 6 
\end{pmatrix}
\begin{pmatrix} P_{1} \\
P_{2} \\
P_{3} \\
P_{4} \\
P_{5}
\end{pmatrix} 
$$

Definimos la matriz $A$ conteniendo en sus filas y columnas el tiempo disponible para cada máquina por unidad de producto.

In [76]:
from IPython.core.completer import ProvisionalCompleterWarning
#La nueva matriz A contiene las horas requeridas para cada producto, donde cada fila corresponde a las horas de M1, M2, M3 y a M4.
A = np.array([[1.5,1.0,2.4,1.0,3.0],[1.0,5.0,1.0,3.5,0.5],[1.5,3.0,3.5,1.0,2.0],[2.0,4.0,4.2,4.3,6.0]]) # \in \mathcal{M}_{45}

#Para calcular las horas trabajadas por máquina, definimos
#recibe vector variable decisión y el índice i de la máquina de la cuál se quiere saber el tiempo total que fue usada por cada producto
def tiempototal(x,i):
    v=np.transpose(x)
    u= np.matmul(A,v) #producto matricial, u es de dimensiones 4x1 (4 máquinas)
    return u[i]

print("En el óptimo, la máquina 1 es usada un total de",tiempototal(sol.x,0), "horas")
print("En el óptimo, la máquina 2 es usada un total de",tiempototal(sol.x,1), "horas")
print("En el óptimo, la máquina 3 es usada un total de",tiempototal(sol.x,2), "horas")
print("En el óptimo, la máquina 4 es usada un total de",tiempototal(sol.x,3), "horas")

#Para determinar el tiempo de horas trabajadas por máquina para cada producto
#el índice i recorre la indexación de las máquinas y j la de los productos.
def tiempoparcial(x,i,j):
    assert type(i)==int and type(j)==int
    assert i>=0 and i<5
    assert j>=0 and j<6
    return A[i,j]*x[j]

#por ejemplo:
print("El producto P1 consumió", tiempoparcial(sol.x,0,0), "horas de M1")
print("El producto P2 consumió", tiempoparcial(sol.x,0,1), "horas de M1")
print("El producto P3 consumió", tiempoparcial(sol.x,0,2), "horas de M1")
print("El producto P4 consumió", tiempoparcial(sol.x,0,3), "horas de M1")
print("El producto P5 consumió", tiempoparcial(sol.x,0,4), "horas de M1")


  

En el óptimo, la máquina 1 es usada un total de 1999.9999999999995 horas
En el óptimo, la máquina 2 es usada un total de 2791.2621359223303 horas
En el óptimo, la máquina 3 es usada un total de 2402.9126213592235 horas
En el óptimo, la máquina 4 es usada un total de 4970.873786407767 horas
El producto P1 consumió 218.44660194174767 horas de M1
El producto P2 consumió 266.99029126213605 horas de M1
El producto P3 consumió 349.514563106796 horas de M1
El producto P4 consumió 291.2621359223299 horas de M1
El producto P5 consumió 873.78640776699 horas de M1


Notemos que la suma de los tiempos consumidos por cada producto en $M_{1}$ es igual a las horas totales consumidas por la máquina. \\

 La siguiente tabla resume las horas utilizadas en total por cada máquina, las cuales respetan las restricciones de horas totales disponibles en consonancia con el hecho de que las calculamos en la solución óptima.

| Máquina |Horas totales consumidas por máquina | 
|----------|----------|
| $M_{1}$    | 1999.99  |
| $M_{2}$    | 2791.26   | 
| $M_{3}$   | 2402.91   | 
| $M_{4}$ | 4970.87 |

En el óptimo, la máquina 1 es la única que consume casi por completo sus horas disponibles por semana. Notemos que $M_{1}$ en general consume menos horas que las demás máquinas pues, además de su limitado tiempo disponible semanal, posee horas requeridas por unidad de producto más bajas en general, trabajando así más rápido.


###Parte 2
Se nos pide evaluar los siguientes casos \\


$a)$ Ya no existe la restricción de stock. En este caso nuestro problema se reduce a \\

$$
 (PL) \left \{ 
 \begin{array}{rcrll}
    \max && 1,1P_{1} + 2,1P_{2} + 2,18P_{3} +2,21P_{4} + 4,4P_{5} \\
    \text{s.a.}  
    &&    4,14P_{1} +5,2P_{2} + 6,66P_{3} + 1,97P_{4} + 5,03P_{5}  &\leq&  5000 \\
    &&    1,5P_{1}+P_{2}+2,4P_{3}+P_{4} + 3P_{5}  &\leq&  2000 \\
    &&    P_{1}+ 5P_{2} + P_{3} + 3,5P_{4} + 0,5P_{5} &\leq& 8000 \\
    &&    1,5P_{1}+3P_{2} + 3,5P_{3} + P_{4}+2P_{5} &\leq& 5000 \\
    &&    2P_{1}+4P_{2} + 4,2P_{3}+ 4,3P_{4} + 6P_{5} &\leq& 12000 \\
    &&    P_{i}&\geq& 0,\ \forall i=1,\dots,5 .&&
 \end{array}
 \right.
 $$

Es decir, sencillamente omitimos la restricción de stock.

In [77]:
#para esta parte podemos usar linprog saltándonos la parte de definir la restriccion de stock también, pero usemos gurobi
from gurobipy import *

m = Model("PL")

#declaramos la variable como continua
p1=m.addVar(vtype=GRB.CONTINUOUS, name='p1')
p2=m.addVar(vtype=GRB.CONTINUOUS, name='p2')
p3=m.addVar(vtype=GRB.CONTINUOUS, name='p3')
p4=m.addVar(vtype=GRB.CONTINUOUS, name='p4')
p5=m.addVar(vtype=GRB.CONTINUOUS, name='p5')

m.update() #Para actualizar el modelo

p=[p1,p2,p3,p4,p5]

def espositivo(P):
  for i in range(len(P)):
    return P[i]>=0
              

m.setObjective(1.1*p1+2.1*p2 +2.18*p3+2.21*p4+4.4*p5, GRB.MAXIMIZE)
m.addConstr(4.14*p1 + 5.2*p2+ 6.66*p3+ 1.97*p4+ 5.03*p5<=5000,'r0')
m.addConstr(p1+5*p2+p3+3.5*p4+0.5*p5<=8000,'r2')
m.addConstr(1.5*p1+p2+2.4*p3+p4+3*p5<=2000,'r1')
m.addConstr(1.5*p1 + 3*p2 + 3.5*p3 + p4 + 2*p5 <=5000,'r3')
m.addConstr(2*p1 + 4*p2 + 4.2*p3 + 4.3*p4 + 6*p5<= 12000,'r4')
m.addConstr(espositivo(p),'r5')

m.update() #actualizamos la variable

m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 5 columns and 26 nonzeros
Model fingerprint: 0xc6243504
Coefficient statistics:
  Matrix range     [5e-01, 7e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 1e+04]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 5 rows, 5 columns, 25 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.6091371e+03   3.904970e+02   0.000000e+00      0s
       4    4.4200000e+03   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.420000000e+03


In [78]:
print("Función objetivo:", m.ObjVal)
print("Para alcanzar la máxima utilidad hay que producir", int(p1.X), "unidades de P1,", int(p2.X), "unidades de P2,", int(p3.X), "unidades de P3,", int(p4.X), "unidades de P4 y",int(p5.X), "unidades de P5.")

Función objetivo: 4420.0
Para alcanzar la máxima utilidad hay que producir 0 unidades de P1, 0 unidades de P2, 0 unidades de P3, 2000 unidades de P4 y 0 unidades de P5.


| Producto | Cantidad óptima a producir | 
|----------|----------|
| $P_{1}$    | 0   |
| $P_{2}$    | 0   | 
| $P_{3}$   | 0   | 
| $P_{4}$ | 2000 |
|$P_{5}$|  0 |

Y la función objetivo alcanza su máximo que, bajo las restricciones impuestas, es igual a 4420.

En contraste con el caso inicial, como el problema de producción ahora carece de restricción de stock, se devela que claramente dadas las horas disponibles de cada máquina y los costos versus precios de los productos, conviene sólo producir el bien $P_{4}$, con el efecto colateral de que se pierde diversidad en la venta de productos.

$b)$ El presupuesto máximo a usar ahora es 4000

In [79]:
#Sólo basta cambiar la primera fila de b, que contiene el presupuesto. El tiempo disponible semanal permanece igual
c = [-1.1,-2.1,-2.18,-2.21,-4.4] #vector de costos
A = [[4.14,5.2,6.66,1.97,5.03],[1.5,1.0,2.4,1.0,3.0],[1.0,5.0,1.0,3.5,0.5],[1.5,3.0,3.5,1.0,2.0],[2.0,4.0,4.2,4.3,6.0]] #matriz de 5x5

#nos falta definir la restricción P_{i} \leq P_{j} \forall j \neq i
#definamos una lista inicialmente vacía
stock = []
for i in range(5):
    for j in range(5):
      if i !=j:
        p=np.zeros(5)
        p[i]=1
        p[j]=-2 #esto es [0,1,-2,0,0] por ej, lo cual es equivalente a decir 1p_{1}<=2p_{2}
        stock += [p]

#concatenamos 
A= A + stock
#donde la matriz stock tiene dimensiones 20x5 (pues son 20 iteraciones)
#luego, A* P = b, A \in \mathcal(M)_{25x5}, P \in \mathcal(M)_{5x1} entonces b \in mathcal(M)_{25x1}
b= np.zeros((25,1))
#asignamos
b[0]=4000
b[1]=2000
b[2]=8000
b[3]=5000
b[4]=12000

#en las demás filas de b habrán 0's, en consonancia con que cada fila verifique p_{i} - 2p_{j}<= 0

P1_bound=(0, None)
P2_bound=(0, None)
P3_bound=(0, None)
P4_bound=(0, None)
P5_bound=(0, None) 

#el vector de costos y los límites de P1, P2, P3, P4 y P5 son los mismos que el caso inicial.
sol = linprog(c, A, b, bounds=[P1_bound,P2_bound,P3_bound, P4_bound,P5_bound])
print("La solución es", -sol.fun)
print(sol.x)

La solución es 2480.0
[133.33333333 133.33333333 133.33333333 266.66666667 266.66666667]


Entonces, las cantidad óptimas a producir son, aproximadamente:

| Producto | Cantidad óptima a producir | 
|----------|----------|
| $P_{1}$    | 133.33   |
| $P_{2}$    | 133.33  | 
| $P_{3}$   | 133.33   | 
| $P_{4}$ | 267.66 |
|$P_{5}$|  267.66 |

Y la función optimizada es igual a 2480. Intuitivamente, este valor óptimo es menor al del caso inicial, pues se dispone de menor presupuesto para invertir, lo cual se traduce en una producción más limitada.

$c)$ El tiempo total disponible por semana de todas las máquinas no debe superar las 27000 horas, mientras que el tiempo individual de cada máquina no puede superar las 13500 horas. \\


| Tipo de máquina | Horas disponibles por semana | 
|----------|----------|
| $M_{1}$    | 13500   |
| $M_{2}$    | 13500   | 
| $M_{3}$   | 13500   | 
| $M_{4}$ | 13500 |
| Total  | 27000 |

De modo que el problema a resolver es


$$
 (PL) \left \{ 
 \begin{array}{rcrll}
    \max && 1,1P_{1} + 2,1P_{2} + 2,18P_{3} +2,21P_{4} + 4,4P_{5} \\
    \text{s.a.}  
    &&    4,14P_{1} +5,2P_{2} + 6,66P_{3} + 1,97P_{4} + 5,03P_{5}  &\leq&  5000 \\
    &&    1,5P_{1}+P_{2}+2,4P_{3}+P_{4} + 3P_{5}  &\leq&  13500 \\
    &&    P_{1}+ 5P_{2} + P_{3} + 3,5P_{4} + 0,5P_{5} &\leq& 13500 \\
    &&    1,5P_{1}+3P_{2} + 3,5P_{3} + P_{4}+2P_{5} &\leq& 13500 \\
    &&    2P_{1}+4P_{2} + 4,2P_{3}+ 4,3P_{4} + 6P_{5} &\leq& 13500 \\
    && \sum_{i=1}^{4} H_{M_{i}} & \leq & 27000 \\
    &&    P_{i}&\leq& 2P_{j},\ \forall i \neq j &&\\
    &&    P_{i}&\geq& 0,\ \forall i=1,\dots,5 .
 \end{array}
 \right.
 $$

 Para expresar la nueva restricción de horas en función de las variables de decisión, podemos definir las horas totales requeridas por unidad de producto. Es decir

 $$
\sum_{i=1}^{4} H_{M_{i}}= 6P_{1}+13P_{2}+11,1P_{3}+12,3P_{4}+11,5P_{5}.
 $$ 


In [80]:
c = [-1.1,-2.1,-2.18,-2.21,-4.4] #vector de costos
A = [[4.14,5.2,6.66,1.97,5.03],[1.5,1.0,2.4,1.0,3.0],[1.0,5.0,1.0,3.5,0.5],[1.5,3.0,3.5,1.0,2.0],[2.0,4.0,4.2,4.3,6.0],[6,13,11.1,9.8,11.5]] #matriz de 6x5

#nos falta definir la restricción P_{i} \leq P_{j} \forall j \neq i
#definamos una lista inicialmente vacía
stock = []
for i in range(5):
    for j in range(5):
      if i !=j:
        p=np.zeros(5)
        p[i]=1
        p[j]=-2 #esto es [0,1,-2,0,0] por ej, lo cual es equivalente a decir 1p_{1}<=2p_{2}
        stock += [p]

#concatenamos 
A= A + stock  
#donde la matriz stock tiene dimensiones 20x5 (pues son 20 iteraciones)
#luego, A* P = b, A \in \mathcal(M)_{25x5}, P \in \mathcal(M)_{5x1} entonces b \in mathcal(M)_{25x1}
b= np.zeros((26,1))
#asignamos
b[0]=5000 #presupuesto
b[1]=13500 #tiempo semanal M_{1}
b[2]=13500 #tiempo semanal M_{2}
b[3]=13500 #tiempo semanal M_{3}
b[4]=13500 #tiempo semanal M_{4}
b[5]=27000 #


P1_bound=(0, None)
P2_bound=(0, None)
P3_bound=(0, None)
P4_bound=(0, None)
P5_bound=(0, None) 

sol = linprog(c, A, b, bounds=[P1_bound,P2_bound,P3_bound, P4_bound,P5_bound])
print("La solución es", -sol.fun)
print("Las cantidades óptimas a producir son", sol.x)


La solución es 3100.0
Las cantidades óptimas a producir son [166.66666667 166.66666667 166.66666667 333.33333333 333.33333333]


Entonces, las cantidades óptimas a producir son, aproximadamente:

| Producto | Cantidad óptima a producir | 
|----------|----------|
| $P_{1}$    | 166.67   |
| $P_{2}$    | 166.67  | 
| $P_{3}$   | 166.67   | 
| $P_{4}$ | 333.33 |
|$P_{5}$|  333.33 |

Y la función optimizada es igual a 3100. En comparación al caso inicial, se tiene una mayor utilidad y se mantiene la diversidad en la producción, por lo que el aumento en las horas disponibles son ventajosas, pese a la restricción de stock.

$d)$ Todas las anteriores incluidas:
Nuevamente, usemos la librería gurobi, y el problema es

$$
 (PL) \left \{ 
 \begin{array}{rcrll}
    \max && 1,1P_{1} + 2,1P_{2} + 2,18P_{3} +2,21P_{4} + 4,4P_{5} \\
    \text{s.a.}  
    &&    4,14P_{1} +5,2P_{2} + 6,66P_{3} + 1,97P_{4} + 5,03P_{5}  &\leq&  4000 \\
    &&    1,5P_{1}+P_{2}+2,4P_{3}+P_{4} + 3P_{5}  &\leq&  13500 \\
    &&    P_{1}+ 5P_{2} + P_{3} + 3,5P_{4} + 0,5P_{5} &\leq& 13500 \\
    &&    1,5P_{1}+3P_{2} + 3,5P_{3} + P_{4}+2P_{5} &\leq& 13500 \\
    &&    2P_{1}+4P_{2} + 4,2P_{3}+ 4,3P_{4} + 6P_{5} &\leq& 13500 \\
    && 6P_{1}+13P_{2}+11,1P_{3}+12,3P_{4}+11,5P_{5} &\leq& 27000\\
    &&    P_{i}&\geq& 0,\ \forall i=1,\dots,5 &&
 \end{array}
 \right.
 $$

In [81]:
from gurobipy import *

n = Model("PL")

#declaramos la variable como continua
p1=n.addVar(vtype=GRB.CONTINUOUS, name='p1')
p2=n.addVar(vtype=GRB.CONTINUOUS, name='p2')
p3=n.addVar(vtype=GRB.CONTINUOUS, name='p3')
p4=n.addVar(vtype=GRB.CONTINUOUS, name='p4')
p5=n.addVar(vtype=GRB.CONTINUOUS, name='p5')

n.update() #Para actualizar el modelo

p=[p1,p2,p3,p4,p5]

def espositivo(P):
  for i in range(len(P)):
    return P[i]>=0
              
n.setObjective(1.1*p1+2.1*p2 +2.18*p3+2.21*p4+4.4*p5, GRB.MAXIMIZE) #maximizar
n.addConstr(4.14*p1 + 5.2*p2+ 6.66*p3+ 1.97*p4+ 5.03*p5<=4000,'r0') #el presupuesto es 4000
n.addConstr(p1+5*p2+p3+3.5*p4+0.5*p5<=13500,'r2') #tiempo disponible M_{2}
n.addConstr(1.5*p1+p2+2.4*p3+p4+3*p5<=13500,'r1') #tiempo disponible M_{1}
n.addConstr(1.5*p1 + 3*p2 + 3.5*p3 + p4 + 2*p5 <=13500,'r3') #tiempo disponible M_{3}
n.addConstr(2*p1 + 4*p2 + 4.2*p3 + 4.3*p4 + 6*p5<= 13500,'r4') #tiempo disponible M_{4}
n.addConstr(espositivo(p),'r5')
n.addConstr(6*p1 +13*p2 +11.1*p3 +12.3*p4 +11.5*p5<=27000,'r6') #restriccion horas totales
#no hay restricción de stock

n.update() #actualizamos la variable

n.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 5 columns and 31 nonzeros
Model fingerprint: 0x428373ea
Coefficient statistics:
  Matrix range     [5e-01, 1e+01]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+03, 3e+04]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 5 columns, 15 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.4873096e+03   0.000000e+00   0.000000e+00      0s
       0    4.4873096e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.487309645e+03


In [82]:
print("Función objetivo:", n.ObjVal)
print("Para alcanzar la máxima utilidad hay que producir", p1.X, "unidades de P1,", p2.X, "unidades de P2,", p3.X, "unidades de P3,", p4.X, "unidades de P4 y",p5.X, "unidades de P5.")

Función objetivo: 4487.309644670051
Para alcanzar la máxima utilidad hay que producir 0.0 unidades de P1, 0.0 unidades de P2, 0.0 unidades de P3, 2030.4568527918782 unidades de P4 y 0.0 unidades de P5.


Finalmente, aunando todas las restricciones anteriores, se obtiene una mayor utilidad respecto al caso inicial a pesar de disponer de un presupuesto menor. Sin embargo, lo óptimo es solo producir unidades $P_{4}$. La siguiente tabla resume los resultados


| Producto | Cantidad óptima a producir | 
|----------|----------|
| $P_{1}$    | 0   |
| $P_{2}$    | 0  | 
| $P_{3}$   | 0   | 
| $P_{4}$ | 2030.45 |
|$P_{5}$|  0 |

Y la función optimizada es aproximadamente 4487,31. \\
Es directo notar que sin la restricción de stock, la solución se carga en $P_{4}$, provocando un desbalance. Por el otro lado, si se evalúa el modelo sólo en materias económicas sencillas (vale decir, no existen aristas de otra índole involucradas en el problema, y descartando la existencia de preferencias de un producto por sobre otro), este caso es el que mayor utilidad genera, y por tanto, es el más conveniente.